In [ ]:
#|default_exp data.features

# Featurizing Time Series

>Functions used to transform time series into a dataframe that can be used to create tabular dataloaders.

In [ ]:
#|export
from tsai.imports import *
from tsai.utils import *
from tsai.data.external import *
from tsai.data.core import *
from tsai.data.tabular import *

In this case we are using tsfresh that is one of the most widely known libraries used to create features from time series. You can get more details about this library here: https://tsfresh.readthedocs.io/en/latest/

In [ ]:
#|export
def get_ts_features(X:Union[np.ndarray, torch.Tensor], y:Union[None, np.ndarray, torch.Tensor]=None, features:Union[str, dict]='min', 
                    n_jobs:Optional[int]=None, **kwargs):
    """
    Args:
        X: np.array or torch.Tesnor of shape [samples, dimensions, timesteps].
        y: Not required for unlabeled data. Otherwise, you need to pass it.
        features: 'min', 'efficient', 'all', or a dictionary. Be aware that 'efficient' and 'all' may required substantial memory and time.
    """    
    
    try: 
        from tsfresh import extract_features
        from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
    except ImportError:
        print("You need to install tsfresh to be able to import tsai.data.features")
    
    df = to_tsfresh_df(X)
    n_jobs = ifnone(n_jobs, defaults.cpus)
    if 'default_fc_parameters' in kwargs.keys(): default_fc_parameters = default_fc_parameters 
    elif features == 'min': default_fc_parameters = MinimalFCParameters()
    elif features == 'efficient': default_fc_parameters = EfficientFCParameters()
    elif features == 'all': default_fc_parameters = ComprehensiveFCParameters()
    else: default_fc_parameters = None
    df = extract_features(df, column_id="id", n_jobs=n_jobs, default_fc_parameters=default_fc_parameters, **kwargs)
    if y is not None:
        if y.ndim == 1: y = y.reshape(-1,1)
        for i in range(y.shape[-1]):
            df['target' if y.shape[-1] == 1 else f'target_{i}'] = y[:, i]
    return df

In [ ]:
#|extras
dsid = 'NATOPS'
X, y, splits = get_UCR_data(dsid, return_split=False)
X.shape

(360, 24, 51)

There are 3 levels of fatures you can extract: 'min', 'efficient' and 'all'. I'd encourage you to start with min as feature creation may take a long time. 

In addition to this, you can pass a dictionary to build the desired features (see tsfresh documentation in the link above).

In [ ]:
#|extras
ts_features_df = get_ts_features(X, y)
ts_features_df.shape

Feature Extraction: 100%|█████████████████████████████████████| 40/40 [00:14<00:00,  2.82it/s]
/Users/nacho/opt/anaconda3/envs/py37torch110/lib/python3.7/site-packages/ipykernel_launcher.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


(360, 241)

The 'min' set creates a dataframe with 8 features per channel + 1 per target (total 193) for each time series sample (360).

In [ ]:
#|extras
cont_names = ts_features_df.columns[:-1]
y_names = 'target'
dls = get_tabular_dls(ts_features_df, splits=splits, cont_names=cont_names, y_names=y_names)
dls.show_batch()

,0__sum_values,0__median,0__mean,0__length,0__standard_deviation,0__variance,0__root_mean_square,0__maximum,0__absolute_maximum,0__minimum,1__sum_values,1__median,1__mean,1__length,1__standard_deviation,1__variance,1__root_mean_square,1__maximum,1__absolute_maximum,1__minimum,2__sum_values,2__median,2__mean,2__length,2__standard_deviation,2__variance,2__root_mean_square,2__maximum,2__absolute_maximum,2__minimum,3__sum_values,3__median,3__mean,3__length,3__standard_deviation,3__variance,3__root_mean_square,3__maximum,3__absolute_maximum,3__minimum,4__sum_values,4__median,4__mean,4__length,4__standard_deviation,4__variance,4__root_mean_square,4__maximum,4__absolute_maximum,4__minimum,5__sum_values,5__median,5__mean,5__length,5__standard_deviation,5__variance,5__root_mean_square,5__maximum,5__absolute_maximum,5__minimum,6__sum_values,6__median,6__mean,6__length,6__standard_deviation,6__variance,6__root_mean_square,6__maximum,6__absolute_maximum,6__minimum,7__sum_values,7__median,7__mean,7__length,7__standard_deviation,7__variance,7__root_mean_square,7__maximum,7__absolute_maximum,7__minimum,8__sum_values,8__median,8__mean,8__length,8__standard_deviation,8__variance,8__root_mean_square,8__maximum,8__absolute_maximum,8__minimum,9__sum_values,9__median,9__mean,9__length,9__standard_deviation,9__variance,9__root_mean_square,9__maximum,9__absolute_maximum,9__minimum,10__sum_values,10__median,10__mean,10__length,10__standard_deviation,10__variance,10__root_mean_square,10__maximum,10__absolute_maximum,10__minimum,11__sum_values,11__median,11__mean,11__length,11__standard_deviation,11__variance,11__root_mean_square,11__maximum,11__absolute_maximum,11__minimum,12__sum_values,12__median,12__mean,12__length,12__standard_deviation,12__variance,12__root_mean_square,12__maximum,12__absolute_maximum,12__minimum,13__sum_values,13__median,13__mean,13__length,13__standard_deviation,13__variance,13__root_mean_square,13__maximum,13__absolute_maximum,13__minimum,14__sum_values,14__median,14__mean,14__length,14__standard_deviation,14__variance,14__root_mean_square,14__maximum,14__absolute_maximum,14__minimum,15__sum_values,15__median,15__mean,15__length,15__standard_deviation,15__variance,15__root_mean_square,15__maximum,15__absolute_maximum,15__minimum,16__sum_values,16__median,16__mean,16__length,16__standard_deviation,16__variance,16__root_mean_square,16__maximum,16__absolute_maximum,16__minimum,17__sum_values,17__median,17__mean,17__length,17__standard_deviation,17__variance,17__root_mean_square,17__maximum,17__absolute_maximum,17__minimum,18__sum_values,18__median,18__mean,18__length,18__standard_deviation,18__variance,18__root_mean_square,18__maximum,18__absolute_maximum,18__minimum,19__sum_values,19__median,19__mean,19__length,19__standard_deviation,19__variance,19__root_mean_square,19__maximum,19__absolute_maximum,19__minimum,20__sum_values,20__median,20__mean,20__length,20__standard_deviation,20__variance,20__root_mean_square,20__maximum,20__absolute_maximum,20__minimum,21__sum_values,21__median,21__mean,21__length,21__standard_deviation,21__variance,21__root_mean_square,21__maximum,21__absolute_maximum,21__minimum,22__sum_values,22__median,22__mean,22__length,22__standard_deviation,22__variance,22__root_mean_square,22__maximum,22__absolute_maximum,22__minimum,23__sum_values,23__median,23__mean,23__length,23__standard_deviation,23__variance,23__root_mean_square,23__maximum,23__absolute_maximum,23__minimum,target
0,-4.150354,-0.432566,-0.081380,51.0,0.597360,0.356840,0.602878,0.778331,0.861279,-0.861279,-58.627594,-1.552286,-1.149561,51.0,0.601466,0.361761,1.297401,-0.056605,1.699415,-1.699415,-45.035744,-0.893296,-0.883054,51.0,0.181012,0.032765,0.901415,-0.495560,1.175184,-1.175184,38.219048,0.796074,0.749393,51.0,0.234144,0.054824,0.785120,1.111021,1.111021,0.392384,-38.509396,-1.455609,-0.755086,51.0,1.031962,1.064946,1.278711,0.693374,1.788006,-1.788006,-31.586580,-0.608169,-0.619345,51.0,0.421872,0.177976,0.749376,0.048263,1.458466,-1.45

In [ ]:
#|extras
x_cat, x_cont, yb = first(dls.train)
x_cont[:10]

tensor([[ 0.2804,  0.4747,  0.2804,  ..., -0.4816, -0.3325,  0.0887],
        [-0.3196, -0.4142, -0.3196,  ..., -0.0979, -0.7659,  0.4576],
        [-0.0764, -0.1685, -0.0764,  ..., -0.1120,  0.7963, -0.8724],
        ...,
        [-1.3950, -0.6392, -1.3950,  ...,  0.6344,  1.4236, -1.4065],
        [-0.0188,  0.0513, -0.0188,  ..., -1.4496, -0.2021, -0.0224],
        [-0.3864, -0.1467, -0.3864,  ...,  0.3099,  1.5548, -1.5182]])

In [ ]:
#|eval: false
#|hide
from tsai.export import get_nb_name; nb_name = get_nb_name(locals())
from tsai.imports import create_scripts; create_scripts(nb_name)

<IPython.core.display.Javascript object>

/Users/nacho/notebooks/tsai/nbs/020_data.features.ipynb saved at 2022-11-09 12:51:20
Correct notebook to script conversion! 😃
Wednesday 09/11/22 12:51:23 CET
